In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=1)

In [ ]:
%time
'''
first we test the core/interface extractor, since it is most essential
'''
import sys
sys.path.append('..')
%matplotlib inline
# output workaround, sometimes necessary
#import matplotlib
#matplotlib.use('Agg')
from eden.converter.graph.gspan import gspan_to_eden
from graphlearn.graphlearn import GraphLearnSampler
from eden.graph import Vectorizer
import matplotlib.pyplot as plt
import itertools
from eden.util import fit, random_bipartition_iter , fit_estimator
from sklearn.linear_model import SGDClassifier
import random


# a vectorizer
vectorizer = Vectorizer( complexity=3 )

# select 1st element in an iterator
def unpack(graphs):
    for graphlist in graphs:
        yield graphlist[0]

# positive set contains 2401 elements, of which we use 30% to test of we cen improve them ,
# the rest is used for the oracle
lenpo=int(2401*.3)


sampler_graph,oracle = random_bipartition_iter(gspan_to_eden('bursi.pos.gspan'),.3)

# we create an oracle
#estimator=make_estimator(oracle,gspan_to_eden('bursi.neg.gspan'))
X=vectorizer.transform(oracle)
X_=X.multiply(-1)
estimator= fit_estimator(SGDClassifier(),X,X_)
print 'estimator ok'


In [ ]:
%%time
import numpy as np

# ok we create an iterator over the graphs we want to work with... 
graphs_pos= sampler_graph


# we want to use an increasing part of the test set.. 
percentages=[0.1, 0.25, 0.5, 0.75, 0.9]
percentages=[0.1]


sampler = GraphLearnSampler(radius_list=[1,2],thickness_list=[1,2])
orires=[]
samplres=[]
for perc in percentages:
    
    originals=[]
    samples=[]
    for e in range(tries):
        # we work with count many graphs
        count = int(lenpo*perc)
        # make copy of graphiterator
        # select count random elements
        # triplicate  the count long iterator
        graphs_pos, graphs_pos_ = itertools.tee(graphs_pos)

        # i pick only perc many :) 
        graphs_pos_,zzz = random_bipartition_iter(graphs_pos_, perc )    
        graphs_pos_,graphs_pos__,graphs_pos___ = itertools.tee(graphs_pos_,3)

        # do sampling
        sampler.fit(graphs_pos__, n_jobs=-1, core_interface_pair_remove_threshold=3, interface_remove_threshold=3)

        n_steps=20
        sampling_factor=3 #how many samples per seed
        sampling_interval=np.ceil([n_steps/sampling_factor])
        improved_graphs = sampler.sample(graphs_pos_,
                            same_radius=False,
                            same_core_size=True,
                            select_cip_max_tries=200,
                            batch_size=30,
                            n_steps=n_steps,
                            sampling_interval=sampling_interval,
                            n_jobs=-1,
                            annealing_factor=1.0,
                            probabilistic_core_choice=True)

        #calculate the score of the improved versions
        #calculate score of the originals
        avg_imp=np.mean( [estimator.decision_function(e) for e in vectorizer.transform(unpack(improved_graphs)) ] )
        avg_ori=np.mean( [estimator.decision_function(e) for e in vectorizer.transform(graphs_pos___)] )
        samples.append(avg_imp)
        originals.append(avg_ori)
        
    orires.append(originals)
    samplres.append(samples)

In [ ]:
print "Original:", orires
print "Sampled:", samplres
# ok alle infos sollten jetzt da sein um diesen supergraph zu zeichnen

from graphlearn.utils.draw import plot_charts
plot_charts(originals, data2=samples, xlabel="Training size", ylabel="Score", size=(10,4), log_scale=False)